#### 1 - Localiza cada moeda, extraindo da imagem um array com cada uma das localizações
#### 2 - em cada recorte da imagem original, transforma em B&W, escala o recorte e extrai as features
#### 3 - passa cada um dos recortes para as MLP's e armazena o resultado de cada imagem
#### 4 - Avalia as probabilidades de cada resultado e informa o resultado com maior probabilidade como resposta

In [ ]:
###

In [29]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from skimage.color import rgb2gray
from joblib import dump, load
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


path_images = 'coin_images/train/5-c/'
coin_img = '1-c_2002_10.jpg'

In [2]:
class ImageProcessor:
    def __init__(self, size):
        self.size = size

    # Função para criar descritores de imagens utilizando HOG
    def create_features(self, image_str_img):
        
        # Se a entrada é uma string, assumimos que é um nome de arquivo
        if isinstance(image_str_img, str):
            # Carrega a imagem
            image = imread(image_str_img)
            try:
                # Carrega a imagem
                image = imread(image_str_img)
            except FileNotFoundError:
                print(f"Arquivo {image_str_img} não encontrado.")
                return None
            except OSError:
                print(f"Erro ao ler o arquivo {image_str_img}.")
                return None
            # Caso contrário, assumimos que é uma imagem
        else:
            image = image_str_img

        # Se a imagem não for em escala de cinza, converte para escala de cinza
        if len(image.shape) > 2:
            image = rgb2gray(image)

        # Redimensiona a imagem para um tamanho fixo
        image = resize(image, self.size)

        # Calcula o descritor HOG para a imagem
        fd = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False)

        return fd

    # Função para processar imagens em um diretório e extrair características
    def process_images(self, dir_name):
        features_dict = {}

        # Para cada arquivo no diretório
        for file_name in os.listdir(dir_name):

            # Se o arquivo for uma imagem
            if (file_name.endswith('.png') or file_name.endswith('.jpg')):

                # Extrai características da imagem
                features = self.create_features(dir_name + file_name)

                # Se houve um erro na leitura da imagem, continue para a próxima
                if features is None:
                    continue

                # Armazena as características no dicionário com o nome do arquivo como a chave
                features_dict[file_name] = features
                
        return features_dict


In [3]:
class ModelData:
    def __init__(self, name, path, path_sufix='', test_size=0.2, image_size=(64, 64)) -> None:
        self.name = name
        self.path = path
        self.path_sufix = path_sufix
        self.test_size = test_size
        self.classifier = None
        self.imageProcessor = ImageProcessor(image_size)
        self.__create_model_data()

    def __create_model_data(self):
        features = []
        labels = []

        for label in ['1-r', '5-c', '10-c', '25-c', '50-c']:
            temp_path = os.path.join(self.path, label, self.path_sufix)
            print(temp_path)
            
            features_dict = self.imageProcessor.process_images(temp_path)
            temp_labels = (len(features_dict.keys()) * (' ' + label)).strip().split(' ')
            # Converte o dicionário de características para uma lista, mantendo a mesma ordem dos rótulos.
            temp_features = [features_dict[file_name] for file_name in sorted(features_dict.keys())]

            features.extend(temp_features)
            labels.extend(temp_labels)

        self.features_train, self.features_test, self.labels_train, self.labels_test = train_test_split(features, labels, test_size=self.test_size, random_state=42)

    def train_classifier(self, load_model = False):
        # Cria o classificador SVM
        if load_model:
            clf = load(f'svm_model_{self.name}.joblib')
        else:
            print(f'Creating classifier... {self.name}')
            clf = svm.SVC(gamma='scale')
            # Treina o classificador SVM
            print('Training classifier...')
            clf.fit(self.features_train, self.labels_train)
            # Salva o modelo
            dump(clf, f'svm_model_{self.name}.joblib')

        # Testa o classificador
        labels_pred = clf.predict(self.features_test)

        # Calcula a precisão do classificador
        accuracy = sum(self.labels_test == labels_pred) / len(self.labels_test)
        print('Acurácia:', accuracy)
        self.classifier = clf


In [ ]:
# Inicialize as listas para armazenar as características e os rótulos
features = []
labels = []

# Para cada diretório na pasta atual
for dir_name in [path_images]:#os.listdir('.'):
    if os.path.isdir(dir_name):

        # Para cada arquivo no diretório
        for file_name in os.listdir(dir_name):

            # Se o arquivo for uma imagem
            if (file_name.endswith('.png') or file_name.endswith('.jpg')) and file_name.endswith('_gs.jpg'):

                # Carregue a imagem em escala de cinza
                img_path = os.path.join(dir_name, file_name)
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                # Redimensione a imagem para 64x64 pixels
                image_resized = cv2.resize(image, (128, 128))

                # Achate a imagem em um vetor
                feature = image_resized.flatten()

                # Normalize as características para terem uma média de 0 e um desvio padrão de 1
                feature = (feature - np.mean(feature)) / np.std(feature)

                # Adicione as características e o rótulo às suas respectivas listas
                features.append(feature)
                labels.append(dir_name)  # assumindo que o nome do diretório é o rótulo

# Converta as listas em arrays numpy
features = np.array(features)
labels = np.array(labels)


In [ ]:
# Inicialize um dicionário para armazenar os SOMs e seus rótulos
soms = {}
som_labels = {}
neurons = 50

# Obtenha os valores únicos de moedas
coin_values = np.unique(labels)

# Para cada valor de moeda
for value in coin_values:

    # Crie um novo SOM
    som = MiniSom(x=neurons, y=neurons, input_len=features.shape[1])

    # Treine o SOM com as características das moedas desse valor
    som.train_random(features[labels == value], num_iteration=5000)

    # Armazene o SOM
    soms[value] = som

    # Inicialize uma matriz para armazenar os rótulos do SOM
    som_labels[value] = np.empty((neurons, neurons), dtype=str)  # Mude para dtype=str

    # Para cada neurônio no SOM
    for i in range(neurons):
        for j in range(neurons):

            # Encontre as características que são mais semelhantes a esse neurônio
            min_dist = np.inf
            min_label = None
            for k, feature in enumerate(features[labels == value]):
                dist = np.linalg.norm(som.get_weights()[i, j] - feature)
                if dist < min_dist:
                    min_dist = dist
                    min_label = labels[labels == value][k]

            # Rotule o neurônio com a classe de moeda mais comum entre as características mais semelhantes
            som_labels[value][i, j] = min_label



In [ ]:
# Inicialize uma matriz para armazenar os rótulos do SOM
som_labels[value] = np.empty((neurons, neurons), dtype=str)  # Mude para dtype=str

# Para cada neurônio no SOM
for i in range(neurons):
    for j in range(neurons):

        # Encontre as características que são mais semelhantes a esse neurônio
        min_dist = np.inf
        min_label = None
        for k, feature in enumerate(features[labels == value]):
            dist = np.linalg.norm(som.get_weights()[i, j] - feature)
            if dist < min_dist:
                min_dist = dist
                min_label = labels[labels == value][k]

        # Rotule o neurônio com a classe de moeda mais comum entre as características mais semelhantes
        som_labels[value][i, j] = min_label


In [ ]:
def classify_coin(image, soms, som_labels):
    # Redimensione a imagem para 64x64 pixels
    image_resized = cv2.resize(image, (128, 128))

    # Achate a imagem em um vetor
    feature = image_resized.flatten()

    # Normalize as características para terem uma média de 0 e um desvio padrão de 1
    feature = (feature - np.mean(feature)) / np.std(feature)

    # Inicialize uma variável para armazenar a menor distância e a classe correspondente
    min_dist = np.inf
    min_class = None

    # Para cada SOM
    for value, som in soms.items():

        # Encontre o neurônio mais semelhante
        winner = som.winner(feature)

        # Calcule a distância entre a característica e o neurônio vencedor
        dist = np.linalg.norm(som.get_weights()[winner] - feature)


        # Se a distância for menor que a menor distância encontrada até agora
        if dist < min_dist:

            # Atualize a menor distância e a classe correspondente
            min_dist = dist
            min_class = som_labels[value][winner]

    # Retorne a classe com a menor distância
    return min_class


In [ ]:
classify_coin(cv2.imread(path_images + coin_img, cv2.IMREAD_GRAYSCALE), soms, som_labels)

In [ ]:
classify_coin(cv2.imread('coin_images/train/1-c/1-c_1994_4_rot90.jpg', cv2.IMREAD_GRAYSCALE), soms, som_labels)

In [ ]:
classify_coin(cv2.imread('coin_images/train/5-c/5-c_2009_17.jpg', cv2.IMREAD_GRAYSCALE), soms, som_labels)

In [ ]:
som_labels

In [25]:
#Cria os dados para treinamento e teste do modelo com variacoes no tamanho da imagem para o hog descriptor
#coinsData_64 = ModelData('coins_64', 'coin_images/train/', '', 0.2, (64, 64))
coinsData_128 = ModelData('coins_128', 'coin_images/train/', '', 0.2, (128, 128))
coinsData_256 = ModelData('coins_256', 'coin_images/train/', '', 0.2, (256, 256))
coinsData_512 = ModelData('coins_512', 'coin_images/train/', '', 0.2, (512, 512))
#bwData_64 = ModelData('coins_bw_64', 'coin_images/train/', 'bw/', 0.2, (64, 64))
#bwData_128 = ModelData('coins_bw_128', 'coin_images/train/', 'bw/', 0.2, (128, 128))
#bwData_256 = ModelData('coins_bw_256', 'coin_images/train/', 'bw/', 0.2, (256, 256))

#modelData_lst = [coinsData_64, coinsData_128, coinsData_256, bwData_64, bwData_128, bwData_256]
modelData_lst = [coinsData_128, coinsData_256, coinsData_512]

coin_images/train/1-r\
coin_images/train/5-c\
coin_images/train/10-c\
coin_images/train/25-c\
coin_images/train/50-c\
coin_images/train/1-r\
coin_images/train/5-c\
coin_images/train/10-c\
coin_images/train/25-c\
coin_images/train/50-c\
coin_images/train/1-r\
coin_images/train/5-c\
coin_images/train/10-c\
coin_images/train/25-c\
coin_images/train/50-c\


In [26]:
for modelData in modelData_lst:
    print(f'Training model {modelData.name}')
    modelData.train_classifier()
    print()

Training model coins_128
Creating classifier... coins_128
Training classifier...
Acurácia: 0.8805309734513275

Training model coins_256
Creating classifier... coins_256
Training classifier...
Acurácia: 0.9424778761061947

Training model coins_512
Creating classifier... coins_512
Training classifier...
Acurácia: 0.9336283185840708



In [91]:
def detect_coins(image, classifier, image_processor):
    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Suaviza a imagem
    #gray = cv2.medianBlur(gray, 5)

    # Detecta círculos na imagem
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 60, param1=150, param2=250, minRadius=0, maxRadius=0)
    if circles is not None:
        circles = np.uint16(np.around(circles))
    else:
        circles = []

    coinCount = 0
    for i in circles[0,:]:
        # Verifica se o círculo está dentro dos limites da imagem
        if i[0]-i[2] >= 0 and i[1]-i[2] >= 0 and i[0]+i[2] <= gray.shape[1] and i[1]+i[2] <= gray.shape[0]:
            # Obtém a região de interesse da imagem
            roi = gray[i[1]-i[2]:i[1]+i[2], i[0]-i[2]:i[0]+i[2]]

            # Verifica se a ROI não está vazia
            if roi.size > 0:
                # Redimensiona a região de interesse para o tamanho esperado pelo classificador
                roi = cv2.resize(roi, image_processor.size)
                #cv2.imshow('Coins', image)
                #print("ROI: ", roi)
                
                # Extrai as características da região de interesse
                features = image_processor.create_features(roi)

                #print("Features: ", features)

                # Classifica a região de interesse
                label = classifier.predict([features])

                # Desenha o círculo na imagem
                cv2.circle(image, (i[0], i[1]), i[2], (0, 0, 255), 2)

                # Desenha o rótulo na imagem
                cv2.rectangle(image, (i[0] - 30 , i[1] - 25), (i[0] + 25, i[1] + 1), (171, 219, 227), cv2.FILLED)
                cv2.putText(image, label[0].replace('-',''), (i[0] - 20, i[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (168, 129, 50), 1, cv2.LINE_AA, False)
                

                # Incrementa o contador de moedas
                coinCount += 1
    

        # Desenha o contador de moedas na imagem
    textSize, _ = cv2.getTextSize(f'Total de moedas: {coinCount}', cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
    textX = image.shape[1] - textSize[0] - 20
    textY = image.shape[0] - 20
    
    cv2.rectangle(image, (textX - 8, textY - textSize[1] - 8), (textX + textSize[0] + 8, textY + 8), (171, 219, 227), cv2.FILLED)
    cv2.putText(image, f'Total de moedas: {coinCount}', (textX, textY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (168, 129, 50), 1, cv2.LINE_AA, False)

    # Mostra a imagem
    cv2.imshow('Detected Coins', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [28]:
# Carrega a imagem
image = cv2.imread('coin_images/test/test_3.jpg')

# Carrega o classificador SVM
classifier = coinsData_512.classifier

# Cria o processador de imagem
image_processor = coinsData_512.imageProcessor

# Detecta as moedas na imagem
detect_coins(image, classifier, image_processor)


In [96]:
# Cria o classificador MLP. 
# A lista [64, 64] define duas camadas ocultas com 64 neurônios cada.
# A função de ativação 'relu' é a função padrão.
# A taxa máxima de iterações (max_iter) é definida como 2000.
mlp = MLPClassifier(hidden_layer_sizes=[128, 1024, 64], activation='logistic', max_iter=10000)

# Treina o classificador MLP.
mlp.fit(coinsData_256.features_train, coinsData_256.labels_train)
print(mlp.score(coinsData_256.features_test, coinsData_256.labels_test))



0.9646017699115044


In [97]:

# Faz a previsão para uma nova instância.
#prediction = mlp.predict([new_instance])
image = cv2.imread('coin_images/test/test_3.jpg')
detect_coins(image, mlp, coinsData_256.imageProcessor)
